In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['movie-web']
collection = db['movieStat']
cursor = collection.find()
list_cur = []
for i in cursor:
    list_cur.append(i)
movies = pd.DataFrame(list_cur)
movies.fillna(0, inplace=True)
movies

,_id,views,comments,numRate,rating
0,633dd3e4f611a788116b43e7,3.0,1.0,2,2.0
1,633dc3b4f611a788116b43d8,2.0,2.0,2,5.0
2,633dc52cf611a788116b43dc,0.0,4.0,3,4.0
3,6354ebb6c834f95698ef02ca,0.0,0.0,1,1.0


In [3]:
movies['rates'] = movies['numRate']*movies['rating']
movies.head()

,_id,views,comments,numRate,rating,rates
0,633dd3e4f611a788116b43e7,3.0,1.0,2,2.0,4.0
1,633dc3b4f611a788116b43d8,2.0,2.0,2,5.0,10.0
2,633dc52cf611a788116b43dc,0.0,4.0,3,4.0,12.0
3,6354ebb6c834f95698ef02ca,0.0,0.0,1,1.0,1.0


In [4]:
df = movies[['views','comments', 'rates']].apply(lambda x:(x.astype(float) - min(x))/(max(x)-min(x)), axis = 0)
df.head()

,views,comments,rates
0,1.000000,0.25,0.272727
1,0.666667,0.50,0.818182
2,0.000000,1.00,1.000000
3,0.000000,0.00,0.000000


In [5]:
movies['score'] = df['views'].pow(2) + df['comments'].pow(2) +  df['rates'].pow(2)
df2 = movies.sort_values(by = 'score', ascending=False).head(10)
df2 = df2.drop(df2.columns[[1, 2, 3, 4, 5]], axis=1)
df2 = df2.rename({'_id': 'movie'}, axis=1)
import datetime as dt
df2['timestamp'] = dt.datetime.now(dt.timezone.utc)
my_dict = df2.to_dict('records')
my_dict

[{'movie': ObjectId('633dc52cf611a788116b43dc'),
  'score': 2.0,
  'timestamp': Timestamp('2022-10-26 08:32:24.038587+0000', tz='UTC')},
 {'movie': ObjectId('633dc3b4f611a788116b43d8'),
  'score': 1.3638659320477502,
  'timestamp': Timestamp('2022-10-26 08:32:24.038587+0000', tz='UTC')},
 {'movie': ObjectId('633dd3e4f611a788116b43e7'),
  'score': 1.1368801652892562,
  'timestamp': Timestamp('2022-10-26 08:32:24.038587+0000', tz='UTC')},
 {'movie': ObjectId('6354ebb6c834f95698ef02ca'),
  'score': 0.0,
  'timestamp': Timestamp('2022-10-26 08:32:24.038587+0000', tz='UTC')}]

In [209]:
my_col = db['trending']
ids = my_col.insert_many(my_dict)
ids

In [6]:
df2

,movie,score,timestamp
2,633dc52cf611a788116b43dc,2.000000,2022-10-26 08:32:24.038587+00:00
1,633dc3b4f611a788116b43d8,1.363866,2022-10-26 08:32:24.038587+00:00
0,633dd3e4f611a788116b43e7,1.136880,2022-10-26 08:32:24.038587+00:00
3,6354ebb6c834f95698ef02ca,0.000000,2022-10-26 08:32:24.038587+00:00


In [8]:
df2.to_csv(r'C:\Users\english\Documents\FinalProject\api\trending.csv', index=False)

In [10]:
trending_list = pd.read_csv('trending.csv')
trending_list

,movie,score,timestamp
0,633dc52cf611a788116b43dc,2.000000,2022-10-26 08:32:24.038587+00:00
1,633dc3b4f611a788116b43d8,1.363866,2022-10-26 08:32:24.038587+00:00
2,633dd3e4f611a788116b43e7,1.136880,2022-10-26 08:32:24.038587+00:00
3,6354ebb6c834f95698ef02ca,0.000000,2022-10-26 08:32:24.038587+00:00


In [11]:
result = trending_list['movie'].tolist()
result

['633dc52cf611a788116b43dc',
 '633dc3b4f611a788116b43d8',
 '633dd3e4f611a788116b43e7',
 '6354ebb6c834f95698ef02ca']